# Datacamp project

# Predicting the Nutriscore grade of a product

##### Authors :
* Alban Petit
* Gaspard Donada--Vidal
* Ghassen Chaabane
* Moez Ezzeddine
* Nicolas Devatine

<img src="resources/nutriscore.jpg" width="250">

This starting kit requires the following dependencies :
* matplotlib
* numpy
* pandas
* seaborn
* scikit-learn
All these modules can be installed with `pip install -r requirements.txt`

You also need the ramp-workflow that can be installed directly from the GitHub repository :
`python -m pip install https://api.github.com/repos/paris-saclay-cds/ramp-workflow/zipball/master`

In [8]:
import numpy as np
#import problem
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

# General Outline

### 1. Business case
1.1. Introduction <br/>
1.2. Business case <br/>
1.3. Data sources <br/>
1.4. Metric
### 2. Exploratory Data Analysis
2.1. Statistics / Vizualization
### 3. Model to submit
3.1. Workflow <br/>
3.2. Feature Extractor <br/>
3.3 Baseline Classifier <br/>
3.4. Evaluation
### 4. Submitting on RAMP

# 1. Business case

## 1.1. Introduction

## 1.2. Business case

## 1.3. Data sources

## 1.4. Metric

# 2. Exploratory Data Analysis

Load the data

In [36]:
data_train = pd.read_csv('data/data_train.csv.zip', sep='|', compression='zip')

In [37]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234372 entries, 0 to 234371
Data columns (total 7 columns):
_id                 234372 non-null float64
allergens           57080 non-null object
brands              204838 non-null object
categories_tags     234371 non-null object
ingredients_tags    164835 non-null object
nova_group          147087 non-null float64
nutriscore_grade    234372 non-null object
dtypes: float64(2), object(5)
memory usage: 12.5+ MB


In [6]:
data.columns

Index(['_id', 'allergens', 'brands', 'categories_tags', 'ingredients_tags',
       'nova_group', 'nutriscore_grade'],
      dtype='object')

In [70]:
data[["_id","nova_group","nutriscore_grade"]].groupby(["nova_group","nutriscore_grade"]).count()

_id
nova_group nutriscore_grade       
1.0        a                 11233
           b                  3404
           c                  3353
           d                  1529
           e                   919
2.0        a                     8
           b                    12
           c                   708
           d                   896
           e                   508
3.0        a                  7806
           b                  6789
           c                  9991
           d                 14551
           e                  7310
4.0        a                  8855
           b                 15191
           c                 26284
           d                 36385
           e                 28246

# 3. Model to submit
## 3.1. Workflow

The model works as follows :
* A feature extractor is used on the input to perform feature engineering
* The features generated are fed to a model to generate a prediction

<p> During the training, the model will use the labels to correct its behaviour. <br> Then, the model is evaluated over a test set using our metric. </p>

<img src="resources/workflow.png" width="800">

## 3.2. Feature Extractor

In [6]:
%%file submissions/starting_kit/feature_extractor.py

class FeatureExtractor:
    def __init__(self):
        
    def fit(self, X_df, y):
        pass
    
    def transform(self, X_df):
        #TODO

Overwriting submissions/starting_kit/feature_extractor.py


## 3.3. Baseline Classifier

In [7]:
%%file submissions/starting_kit/classifier.py
from sklearn.tree import DecisionTree

class Classifier(BaseEstimator):
    
    def __init__(self):
        self.model = DecisionTree()
        
    def fit(self, X, y):
        self.model.fit(X, y)
        
    def predict(self, X):
        return self.model.predict(X)

Overwriting submissions/starting_kit/classifier.py


# 4. Submitting on RAMP
You can test locally your submission before sending it to the RAMP platform backend using the command !ramp_test_submission --submission starting_kit.

The usual way to use this is to explore various models, feature transformations, hyper-parameters optimization and test them using the following command. It uses our metric to estimate your score.

To test files stored in a different folder, just replace 'starting_kit' by the name of your folder

In [ ]:
!ramp_test_submission --submission starting_kit